<a href="https://colab.research.google.com/github/vaishnavi123-cOde/assignment/blob/main/Weapon_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Jul 18 16:32:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import zipfile, os, shutil
from sklearn.model_selection import train_test_split

# Define paths
zip_path = '/content/data.zip.zip'  # Colab-style path
extract_path = '/content/weapons-dataset'
images_path = os.path.join(extract_path, 'images')
labels_path = os.path.join(extract_path, 'labels')

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Gather all images and corresponding labels
image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))]
label_files = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in image_files]

# Split into train, val, test (70%, 20%, 10%)
train_imgs, test_imgs = train_test_split(image_files, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(test_imgs, test_size=1/3, random_state=42)  # 0.2 val, 0.1 test

# Helper function to copy files
def copy_files(img_list, dest_img_dir, dest_label_dir):
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)
    for img_name in img_list:
        label_name = img_name.replace('.jpg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(images_path, img_name), dest_img_dir)
        shutil.copy(os.path.join(labels_path, label_name), dest_label_dir)

# Organize directories
base = '/content/yolo_gun_dataset'
copy_files(train_imgs, f'{base}/train/images', f'{base}/train/labels')
copy_files(val_imgs, f'{base}/val/images', f'{base}/val/labels')
copy_files(test_imgs, f'{base}/test/images', f'{base}/test/labels')


In [ ]:
yaml_content = """
path: /content/yolo_gun_dataset
train: train/images
val: val/images
test: test/images

names:
  0: gun
"""

with open('/content/gun_data.yaml', 'w') as f:
    f.write(yaml_content)


In [ ]:
from ultralytics import YOLO

# Load a YOLOv8 model (or use a custom one)
model = YOLO('yolov8n.pt')  # or yolov8s.pt, yolov8m.pt depending on your setup

# Train the model
model.train(data='/content/gun_data.yaml', epochs=50, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 358MB/s]

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gun_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, save=Tr

100%|██████████| 755k/755k [00:00<00:00, 117MB/s]

...

                   all        105        126      0.766      0.802      0.797      0.442



50 epochs completed in 0.100 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.09it/s]


                   all        105        126      0.766      0.802      0.797      0.442
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 4.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c7033a15110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
results = model.predict(source='/content/yolo_gun_dataset/test/images', save=True, conf=0.5)



image 1/53 /content/yolo_gun_dataset/test/images/002e137e-ASP_111_jpeg.rf.f54cc9a0d8a4adada8a5bf20c1747aba.jpg: 640x640 (no detections), 26.1ms
image 2/53 /content/yolo_gun_dataset/test/images/03d30d68-115_jpeg.rf.60e1928dd4672c0c3eb218bdd2b9edb3.jpg: 640x640 1 gun, 15.0ms
image 3/53 /content/yolo_gun_dataset/test/images/08d8adec-169_jpeg.rf.4f26969f5d673d4b1ab8181160fbc2c9.jpg: 640x640 1 gun, 40.2ms
image 4/53 /content/yolo_gun_dataset/test/images/0b7858cc-180_jpeg.rf.4bbda5b9a2e60543b1a0011429b44ef4.jpg: 640x640 2 guns, 40.9ms
image 5/53 /content/yolo_gun_dataset/test/images/16623beb-149_jpeg.rf.4429da0b164570f7dda5dc679efdc4fa.jpg: 640x640 1 gun, 11.3ms
image 6/53 /content/yolo_gun_dataset/test/images/1d148356-ASP_152_png.rf.a8c171eef8ab5b2ef85d725f9d686603.jpg: 640x640 1 gun, 16.3ms
image 7/53 /content/yolo_gun_dataset/test/images/1fca3387-324_jpeg.rf.0c837de6617340e3d2f5be8719fcf349.jpg: 640x640 1 gun, 33.3ms
image 8/53 /content/yolo_gun_dataset/test/images/2637e1e5-143_jpeg.rf.e